In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
from urllib.parse import urljoin
import os
from pymysql import Connection

In [2]:
url = "http://10.1.88.249:7000/"
r = requests.get(url)
html = r.text
print(html)

<!DOCTYPE html>
<html lang="en">
<head>
    
    <meta charset="UTF-8">
    <title>文学天地</title>
    <link rel="stylesheet" type="text/css" href='/static/css/index.css' >
    <style>
        #header {
            background-image:url(/static/back_img.jpg);
        }
    </style>
</head>
<body>

<center>
      <div id="all">
            <div id="header">万笔书斋</div>
            
            <div class="book">
                <a class="name" href="/红楼梦"><img class="main" src=/static/%E7%BA%A2%E6%A5%BC%E6%A2%A6.jpg>红楼梦</a>
                <p class="author">作者：曹雪芹</p>
                <p class="price">价格：36.90</p>
            </div>
              
            <div class="book">
                <a class="name" href="/水浒传"><img class="main" src=/static/%E6%B0%B4%E6%B5%92%E4%BC%A0.jpg>水浒传</a>
                <p class="author">作者：施耐庵</p>
                <p class="price">价格：23.50</p>
            </div>
              
            <div class="book">
                <a class="name" href="/庆余年"><img cl

In [3]:
#获取每本书的名字
soup = BeautifulSoup(html, 'html.parser')
i=0
book_intro = []


# 找到每本书的所有a标签和
a_tags = soup.find_all('a')
p1 = soup.find_all('p', _class='author')


# 将每本书的名字存到一个列表里
book_name = []
for a in a_tags:
    book_name.append(a.text)
book_name.pop(2)
print(book_name)


# 找到所有书籍的父级元素
books = soup.find_all('div', class_='book')
books.pop(2)
for book in books:
    # 获取作者名字
    author = book.find('p', class_='author').get_text(strip=True)
    # 获取价格
    price = book.find('p', class_='price').get_text(strip=True)
    book_intro.append([book_name[i],author,price])
    i+=1
# print(book_intro)


['红楼梦', '水浒传', '三国演义']


In [4]:
# # 确定每一本书的网址
full_urls = [urljoin(url,a) for a in book_name]
print(full_urls)

['http://10.1.88.249:7000/红楼梦', 'http://10.1.88.249:7000/水浒传', 'http://10.1.88.249:7000/三国演义']


In [5]:
page = "page"
i = 0
# a = 1

con = None
# 创建数据库连接
con = Connection(
    host="localhost",# 主机名
    port=3306, # 端口
    user="root", # 账户
    passwd="hk123" # 密码
)

# 设置游标
cursor = con.cursor()
# 选择数据库
cursor.execute("use library")
# 提交结果
con.commit()

for url in full_urls:
        
        # 找到每一章内容的URL
        r = requests.get(url)
        html = r.text
        soup = BeautifulSoup(html, 'html.parser')
        a_tag = soup.find_all('a')
        books_url = [urljoin(url, f"{url}/{page}/{i}") for i in range(len(a_tag))]


        # # 如果图书目录不存在，则创建该目录
        # book_directory = os.path.join('C:\\Users\\dell\\Desktop\\PycharmProjects\\学期项目\\2023下\\数据库\\books', book_name[i])
        # if not os.path.exists(book_directory):
        #     os.makedirs(book_directory)


        # 获取内容
        for book_url in books_url:
            try:
                print(book_url)
                # 解析每一章的页面
                r = requests.get(book_url)
                html = r.text
                soup = BeautifulSoup(html, 'html.parser')

                h1_tag = soup.find("h1") # 找到标题
                content_div = soup.find('div', {'id': 'content'}) # 找到内容

                # 使用正则表达式提取数字
                num = re.search(r'/(\d+)$', book_url)
                last_number = num.group(1)

                # 写入数据
                insert_sql = "INSERT INTO book_content (book_name, chapter_num, title, content) VALUES (%s, %s, %s, %s)"
                cursor.execute(insert_sql, (book_name[i], str(int(last_number)+1), h1_tag.text, content_div))
                print(f"{book_name[i]}---{int(last_number)+1}成功！")

            except Exception as e:
                print("输得很彻底！："+str(e))
        i += 1
        

con.commit() # 提交结果
con.close() # 关闭连接

        # 写入txt文件
        #     try:
        #         # 将内容保存到文件中
        #         file_path = os.path.join(book_directory, f"第{a}章.txt")
        #         with open(file_path, 'w', encoding='utf-8') as f:
        #             f.write(h1_tag.text)
        #             f.write(content_div.text)
        #             a += 1
        #     except Exception as e:
        #         print(e)
        # a=1

http://10.1.88.249:7000/红楼梦/page/0
红楼梦---1成功！
http://10.1.88.249:7000/红楼梦/page/1
红楼梦---2成功！
http://10.1.88.249:7000/红楼梦/page/2
红楼梦---3成功！
http://10.1.88.249:7000/红楼梦/page/3
红楼梦---4成功！
http://10.1.88.249:7000/红楼梦/page/4
红楼梦---5成功！
http://10.1.88.249:7000/红楼梦/page/5
红楼梦---6成功！
http://10.1.88.249:7000/红楼梦/page/6
红楼梦---7成功！
http://10.1.88.249:7000/红楼梦/page/7
红楼梦---8成功！
http://10.1.88.249:7000/红楼梦/page/8
红楼梦---9成功！
http://10.1.88.249:7000/红楼梦/page/9
红楼梦---10成功！
http://10.1.88.249:7000/红楼梦/page/10
红楼梦---11成功！
http://10.1.88.249:7000/红楼梦/page/11
红楼梦---12成功！
http://10.1.88.249:7000/红楼梦/page/12
红楼梦---13成功！
http://10.1.88.249:7000/红楼梦/page/13
红楼梦---14成功！
http://10.1.88.249:7000/红楼梦/page/14
红楼梦---15成功！
http://10.1.88.249:7000/红楼梦/page/15
红楼梦---16成功！
http://10.1.88.249:7000/红楼梦/page/16
红楼梦---17成功！
http://10.1.88.249:7000/红楼梦/page/17
红楼梦---18成功！
http://10.1.88.249:7000/红楼梦/page/18
红楼梦---19成功！
http://10.1.88.249:7000/红楼梦/page/19
红楼梦---20成功！
http://10.1.88.249:7000/红楼梦/page/20
红楼梦---21成功！
http://10.1

In [6]:
# i=0
# for intro in book_intro:
#     book_directory = os.path.join('C:\\Users\\dell\\Desktop\\PycharmProjects\\学期项目\\2023下\\数据库\\books', a_tags[i])
#     file_path = os.path.join(book_directory, f"简介.txt")
#     with open(file_path, 'w', encoding='utf-8') as f:
#         f.write(str(intro))
#     i+=1
import re

url = "http://10.1.88.249:7000/红楼梦/page/10"

# 使用正则表达式提取数字
match = re.search(r'/(\d+)$', url)

if match:
    last_number = match.group(1)
    print(f"最后一个数字是：{last_number}")
else:
    print("未找到数字")


最后一个数字是：10
